# Week 12 Problem 3

If you are not using the `Assignments` tab on the course JupyterHub server to read this notebook, read [Activating the assignments tab](https://github.com/UI-DataScience/info490-fa16/blob/master/Week2/assignments/README.md).

A few things you should keep in mind when working on assignments:

1. Make sure you fill in any place that says `YOUR CODE HERE`. Do **not** write your answer in anywhere else other than where it says `YOUR CODE HERE`. Anything you write anywhere else will be removed or overwritten by the autograder.

2. Before you submit your assignment, make sure everything runs as expected. Go to menubar, select _Kernel_, and restart the kernel and run all cells (_Restart & Run all_).

3. Do not change the title (i.e. file name) of this notebook.

4. Make sure that you save your work (in the menubar, select _File_ → _Save and CheckPoint_)

5. You are allowed to submit an assignment multiple times, but only the most recent submission will be graded.

In [ ]:
from nose.tools import assert_equal

## Problem 1.

In this problem, we will working on the `movies.csv` and the `ratings.csv` under the `~/data/ml-latest-small/` directory using an SQLite database. We will write SQL scripts using the iPython [`writefile`](https://ipython.org/ipython-doc/3/interactive/magics.html#cellmagic-writefile) magic, and then run bash commands like
```bash
$ sqlite3 problem3.db < script.sql
```
in which `problem3.db` is our database and `script.sql` is the script, to execute the SQL commands in the script. Let's first create two tables named `movies` and `ratings` from those two files. Here I'll do this for you.

First, we create [sym links](https://en.wikipedia.org/wiki/Symbolic_link#POSIX_and_Unix-like_operating_systems) in the current directory to save our efforts calling the directories.

In [ ]:
MOVIES_PATH = "/home/data_scientist/data/ml-latest-small/movies.csv"
RATINGS_PATH = "/home/data_scientist/data/ml-latest-small/ratings.csv"

!ln -sf $MOVIES_PATH movies.csv
!ln -sf $RATINGS_PATH ratings.csv

Below is an sql script that creates a table named `movies` from the `movies.csv` file, and then remove the header line and display the first 5 rows. We write this script to a file named `movies.sql` using the iPython `writefile` magic.

In [ ]:
%%writefile movies.sql

-- Create the movies table from our familiar movies.csv file.
DROP TABLE IF EXISTS movies;

CREATE TABLE movies(
    movieId INT,
    title TEXT,
    genres TEXT
);

.separator ,
.import movies.csv movies

-- The following line removes the header.
DELETE FROM movies WHERE movieId='movieId';

-- Display the first 5 rows.
SELECT * FROM movies LIMIT 5;

For this problem, let's name our database as `problem3.db`. Here, we use the ! magic to redirect the `movies.sql` code to `sqlite3`. When you run the code cell below, you should be able to see the first 5 rows in the `movies` table as the output of the `SELECT` statement. You could find the column names and corresponding data types from the script directly.

In [ ]:
!sqlite3 problem3.db < movies.sql

Following the same process, let's create the `ratings` table from `ratings.csv`, remove the header, and then select the first 5 rows. Again, the column names and data types are shown in the script directly.

In [ ]:
%%writefile ratings.sql

-- Create the ratings table from ratings.csv.
DROP TABLE IF EXISTS ratings;

CREATE TABLE ratings(
    userId INT,
    movieId INT,
    rating REAL,
    timestamp INT
);

.separator ,
.import ratings.csv ratings

-- The following line removes the header.
DELETE FROM ratings WHERE userId='userId';

-- Display the first 5 rows.
SELECT * FROM ratings LIMIT 5;

In [ ]:
!sqlite3 problem3.db < ratings.sql

## 1. Create the `validRatings` table

In the `ratings` table, each row is a rating record per movie per user. But the average rating for each movie is of more interest. Let's create a new table from the `ratings` table that contains the average rating for each movie. We would like to filter out the movies with few ratings because their avearge rating scores do not actually make sense. 

In the following code cell, write SQL statements into the sql script that create a table named `validRatings` following the instructions below. **Don't be scared by the long instruction. This is a simple operation as long as you read and follow the instruction.**
- The iPython magic to write the script is provided (for every problem in this notebook). You only need to write SQL commands.
- It's strongly recommended to start by writing "**`DROP TABLE IF EXISTS validRatings;`**" so that SQL will not complain if you try to recreate the table.
- We are going to create a new table (`validRatings`) from an exisiting table (`ratings`). You should use "**`CREATE TABLE newTableName AS SELECT ... FROM ...`**" so that you could apply a `SELECT` statement and aggregate functions.
- Create a table named `validRatings` from the `ratings` table, which has three columns: **`movieId`, `averageRating`, `ratingCounts`**. The first column `movieId` is copied from the `movieId` column in the `ratings` table. The second column is the average of the `rating` column in the `ratings` table (for each `movieId`) . The last column is the count of the `rating` column (for each `movieId`). 
- You should have a [**`GROUP BY`**](http://www.w3schools.com/sql/sql_groupby.asp) clause in your `SELECT` statement to make sure the aggregate functions are applied for each `movieId`. Without that clause, the average function will calculate the average of all rows, and so will the count function. 
- Finally, you should only select the rows with rating counts larger than 30. You should use a [**`HAVING`**](http://www.w3schools.com/sql/sql_having.asp) clause which allows you to include aggregate functions as the filtering condition. The idea of `HAVING` is very similar to `WHERE`. Note that a `WHERE` clause will not work, because the `WHERE` keyword could not be used with aggregate functions. 

In [ ]:
%%writefile ratings_operations.sql

-- YOUR CODE HERE

Let's execute your SQL script and display the first 5 rows. You should be able to see:
```
1|3.9073275862069|232
2|3.35326086956522|92
3|3.18965517241379|58
5|3.25|62
6|4.07391304347826|115
```
If your SQL command has any problem, the SQL database wil give you an error message as the output of the code cell below, which, in general, is human-readable that helps you debug.

In [ ]:
!sqlite3 problem3.db < ratings_operations.sql

In [ ]:
!sqlite3 problem3.db "SELECT * FROM validRatings LIMIT 5;"

In [ ]:
# sql assertion tests

table_exists = !sqlite3 problem3.db "SELECT name FROM sqlite_master WHERE type='table' and name='validRatings';"
assert_equal(table_exists.s, "validRatings", msg="There's no table named \"validRatings\" in the database.")

table_info = !sqlite3 problem3.db "PRAGMA table_info(validRatings);"
columns = [t.split("|")[1] for t in table_info]
columns_answer = ['movieId', 'averageRating', 'ratingCounts']
assert_equal(set(columns), set(columns_answer))

total_rows = !sqlite3 problem3.db "SELECT COUNT(*) FROM validRatings;"
assert_equal(total_rows.s, "861")

first_five = !sqlite3 problem3.db "SELECT * FROM validRatings LIMIT 5;"
first_five_answer = ['1|3.9073275862069|232',
                     '2|3.35326086956522|92',
                     '3|3.18965517241379|58',
                     '5|3.25|62',
                     '6|4.07391304347826|115']
assert_equal(set(first_five), set(first_five_answer))

counts = !sqlite3 problem3.db "SELECT ratingCounts FROM validRatings WHERE movieId=497;"
assert_equal(counts.s, "67")

## 2. Join `validRatings` and `movies` into a new table

In the code cell below, write SQL commands to create a table named `movieRatings` by implicitly joining `validRatings` and `movies` tables.

- "**`DROP TABLE IF EXISTS`**" is strongly recommended.
- Again, you should use "**`CREATE TABLE newTableName AS SELECT ... FROM ... **`**" to create a new table from existing tables.
- The common column in both tables is the **`movieId`** column. Use that column to match the two tables.
- The resulting new table `movieRatings` have three columns: **`averageRating`, `title`, `genres`**. The first column `averageRating` is copied from the `validRatings` table, and the other two columns are from the `movies` table.

In [ ]:
%%writefile join.sql

-- YOUR CODE HERE

The first column of the `movieRatings` table is:

```
3.9073275862069|Toy Story (1995)|Adventure|Animation|Children|Comedy|Fantasy
3.35326086956522|Jumanji (1995)|Adventure|Children|Fantasy
3.18965517241379|Grumpier Old Men (1995)|Comedy|Romance
3.25|Father of the Bride Part II (1995)|Comedy
4.07391304347826|Heat (1995)|Action|Crime|Thriller
```

In [ ]:
!sqlite3 problem3.db < join.sql

In [ ]:
!sqlite3 problem3.db "SELECT * FROM movieRatings LIMIT 5;"

In [ ]:
# sql assertion tests

movieRatings_exists = !sqlite3 problem3.db "SELECT name FROM sqlite_master WHERE type='table' and name='movieRatings';"
assert_equal(movieRatings_exists.s, "movieRatings", msg="There's no table named \"movieRatings\" in the database.")

movieRatings_info = !sqlite3 problem3.db "PRAGMA table_info(movieRatings);"
movieRatings_columns = [t.split("|")[1] for t in movieRatings_info]
movieRatings_columns_answer = ['averageRating', 'title', 'genres']
assert_equal(set(movieRatings_columns), set(movieRatings_columns_answer))

movieRatings_total_rows = !sqlite3 problem3.db "SELECT COUNT(*) FROM movieRatings;"
assert_equal(movieRatings_total_rows.s, "861")

movieRatings_first_five = !sqlite3 problem3.db "SELECT * FROM movieRatings LIMIT 5;"
movieRatings_first_five_answer = ['3.9073275862069|Toy Story (1995)|Adventure|Animation|Children|Comedy|Fantasy',
                                  '3.35326086956522|Jumanji (1995)|Adventure|Children|Fantasy',
                                  '3.18965517241379|Grumpier Old Men (1995)|Comedy|Romance',
                                  '3.25|Father of the Bride Part II (1995)|Comedy',
                                  '4.07391304347826|Heat (1995)|Action|Crime|Thriller']
assert_equal(set(movieRatings_first_five), set(movieRatings_first_five_answer))

rating = !sqlite3 problem3.db "SELECT averageRating FROM movieRatings WHERE title='Interstellar (2014)';"
assert_equal(rating.s, "4.06578947368421")

## 3. Find the best horror movies

In the code cell below, write an SQL command to find every row satisfying ALL of these conditions:

- Its `averageRating` is above 4.0;
- Its `genres` contains the string "`Horror`" at any place, i.e. the pattern is "`%Horror%`";
- Order the selected rows by its `averageRating` score in a **descending** order;
- All the columns should be displayed.

In [ ]:
%%writefile best_horror_movies.sql

-- YOUR CODE HERE

When you execute the code cell below, you should get: 
```
4.1948275862069|Silence of the Lambs, The (1991)|Crime|Horror|Thriller
4.18604651162791|Psycho (1960)|Crime|Horror
4.14649681528662|Aliens (1986)|Action|Adventure|Horror|Sci-Fi
4.13089005235602|Sixth Sense, The (1999)|Drama|Horror|Mystery
4.06410256410256|Alien (1979)|Horror|Sci-Fi
4.05|Shaun of the Dead (2004)|Comedy|Horror
4.03125|Battle Royale (Batoru rowaiaru) (2000)|Action|Drama|Horror|Thriller
4.00446428571429|Shining, The (1980)|Horror
```

In [ ]:
!sqlite3 problem3.db < best_horror_movies.sql

In [ ]:
# sql assertion tests

best_horrors_query = !sqlite3 problem3.db < best_horror_movies.sql
best_horrors_query_answer = ['4.1948275862069|Silence of the Lambs, The (1991)|Crime|Horror|Thriller',
                             '4.18604651162791|Psycho (1960)|Crime|Horror',
                             '4.14649681528662|Aliens (1986)|Action|Adventure|Horror|Sci-Fi',
                             '4.13089005235602|Sixth Sense, The (1999)|Drama|Horror|Mystery',
                             '4.06410256410256|Alien (1979)|Horror|Sci-Fi',
                             '4.05|Shaun of the Dead (2004)|Comedy|Horror',
                             '4.03125|Battle Royale (Batoru rowaiaru) (2000)|Action|Drama|Horror|Thriller',
                             '4.00446428571429|Shining, The (1980)|Horror']

assert_equal(len(best_horrors_query), len(best_horrors_query_answer))
assert_equal(set(best_horrors_query), set(best_horrors_query_answer))

## 4. Find the most popular movie

In the code cell below, write an SQL command to find the most popular movie, i.e. the movie with the highest `ratingCounts`.

- You need to apply aggregate functions to find the maximum of `ratingCounts` in the `validRatings` table.
- In addition to the maximum of `ratingCounts`, we would also like to know the movie `title` and `genres`. Those data can be extracted from the `movies` table.
- The output has three columns: **maximum `ratingCounts`, `title`, `genres`**. 
- Since we are using info from two tables, we need to do an implicit join again. The common column is still **`movieId`**.

In [ ]:
%%writefile most_popular.sql

-- YOUR CODE HERE

When the following command is executed, you should be able to see: 
```
325|Pulp Fiction (1994)|Comedy|Crime|Drama|Thriller
```

In [ ]:
!sqlite3 problem3.db < most_popular.sql

In [ ]:
#sql assertion tests

most_popular_query = !sqlite3 problem3.db < most_popular.sql
most_popular_query_answer = "325|Pulp Fiction (1994)|Comedy|Crime|Drama|Thriller"
assert_equal(most_popular_query.s, most_popular_query_answer)

## Clearning up

In [ ]:
!rm problem3.db movies.sql ratings.sql best_horror_movies.sql \
 most_popular.sql movies.csv ratings.csv